In [1]:
import torch
import torchaudio
import datasets
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import re


/sysadmin/wirawan/miniconda3/envs/bert2bert/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
# Preprocessing the datasets.
# We need to read the audio files as arrays
def speech_file_to_array_fn(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
    batch["sentence"] = batch["sentence"].replace('！ ', '')
    batch["sentence"] = batch["sentence"].replace('，', '')
    batch["sentence"] = batch["sentence"].replace('é', 'e')    
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    resampler = torchaudio.transforms.Resample(sampling_rate, 16_000)
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    return batch

In [3]:
# Preprocessing the datasets.
# We need to read the aduio files as arrays
def evaluate(batch):
    inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_strings"] = processor.batch_decode(pred_ids)
    return batch

In [4]:
#dataset = datasets.load_from_disk("sundanese_train_dataset")

In [5]:
test_dataset = load_dataset("common_voice", "id", split="test")
wer = load_metric("wer")

Couldn't find file locally at common_voice/common_voice.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.4.1/datasets/common_voice/common_voice.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/datasets/common_voice/common_voice.py.
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/id/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f)


In [6]:
%%capture

model_name = "cahya/wav2vec2-large-xlsr-indonesian-mix"
#model_name = "./wav2vec2-large-xlsr-indonesian-articial-CV"
#model_name = "cahya/wav2vec2-large-xlsr-indonesian"
#model_name = "ayameRushia/wav2vec2-large-xlsr-indonesia-demo"
#model_name = "Galuh/wav2vec2-large-xlsr-indonesian-demo"

processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name) 
model.to("cuda")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [7]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\'\”]'
test_dataset = test_dataset.map(speech_file_to_array_fn)

Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/id/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-6273e8490b649de6.arrow


In [8]:
result = test_dataset.map(evaluate, batched=True, batch_size=8)

print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["sentence"])))


WER: 19.373485


In [9]:
result = test_dataset.map(evaluate, batched=True, batch_size=64)

print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["sentence"])))


WER: 19.671825


### Check the sampling rate of the sound files

In [28]:
train_dataset = load_dataset("common_voice", "id", split="train+validation")
test_dataset = load_dataset("common_voice", "id", split="test")

Couldn't find file locally at common_voice/common_voice.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.4.1/datasets/common_voice/common_voice.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/datasets/common_voice/common_voice.py.
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/id/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f)
Couldn't find file locally at common_voice/common_voice.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.4.1/datasets/common_voice/common_voice.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/datasets/common_voice/common_voice.py.
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/id/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f)


In [17]:
test_dataset

Dataset({
    features: ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 1844
})

In [18]:
test_dataset[0]

{'accent': '',
 'age': '',
 'client_id': '057bf45c0c338db897f5717f744bcac8a2ac2eee990a4294ce406f1f79c3326aff181d59e49e7e82813e532ed160c027a80eb4c7e36d8899d3fc96e48d102de5',
 'down_votes': 0,
 'gender': '',
 'locale': 'id',
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/fd8a16a97efd77adba3c26c54d0cfae6c9d9494c1017f8070f3f79db72c4b57c/cv-corpus-6.1-2020-12-11/id/clips/common_voice_id_22888800.mp3',
 'segment': "''",
 'sentence': 'Minggu depan kakak perempuan saya menikah.',
 'up_votes': 2}

In [35]:
sampling_rates = {}
for i, data in enumerate(test_dataset):    
    speech_array, sampling_rate = torchaudio.load(data["path"])
    if sampling_rate not in sampling_rates:
        sampling_rates[sampling_rate] = 1
    else:
        sampling_rates[sampling_rate] += 1

In [36]:
for rate in sampling_rates:
    print("Rate", rate, sampling_rates[rate], sampling_rates[rate]/len(test_dataset)*100.0)

Rate 48000 1682 91.21475054229936
Rate 32000 115 6.236442516268981
Rate 44100 44 2.386117136659436


In [33]:
sampling_rates = {}
for i, data in enumerate(train_dataset):    
    speech_array, sampling_rate = torchaudio.load(data["path"])
    if sampling_rate not in sampling_rates:
        sampling_rates[sampling_rate] = 1
    else:
        sampling_rates[sampling_rate] += 1

In [34]:
for rate in sampling_rates:
    print("Rate", rate, sampling_rates[rate], sampling_rates[rate]/len(train_dataset)*100.0)

Rate 48000 3965 100.0


In [31]:
len(train_dataset)

3965

In [32]:
sampling_rates

{48000: 3964}

### Turkish

In [37]:
train_dataset = load_dataset("common_voice", "tr", split="train+validation")
test_dataset = load_dataset("common_voice", "tr", split="test")

Couldn't find file locally at common_voice/common_voice.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.4.1/datasets/common_voice/common_voice.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/datasets/common_voice/common_voice.py.


Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f. Subsequent calls will reuse this data.


Couldn't find file locally at common_voice/common_voice.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.4.1/datasets/common_voice/common_voice.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/datasets/common_voice/common_voice.py.
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/tr/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f)


In [38]:
sampling_rates = {}
for i, data in enumerate(test_dataset):    
    speech_array, sampling_rate = torchaudio.load(data["path"])
    if sampling_rate not in sampling_rates:
        sampling_rates[sampling_rate] = 1
    else:
        sampling_rates[sampling_rate] += 1

In [39]:
for rate in sampling_rates:
    print("Rate", rate, sampling_rates[rate], sampling_rates[rate]/len(train_dataset)*100.0)

Rate 48000 1643 47.23979298447383
Rate 44100 4 0.11500862564692352


In [40]:
sampling_rates = {}
for i, data in enumerate(train_dataset):    
    speech_array, sampling_rate = torchaudio.load(data["path"])
    if sampling_rate not in sampling_rates:
        sampling_rates[sampling_rate] = 1
    else:
        sampling_rates[sampling_rate] += 1

In [41]:
for rate in sampling_rates:
    print("Rate", rate, sampling_rates[rate], sampling_rates[rate]/len(train_dataset)*100.0)

Rate 48000 3462 99.53996549741231
Rate 44100 16 0.46003450258769407
